In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#pip install --upgrade ipykernel

In [ ]:
#pip install autocorrect


In [ ]:
#pip install textblob


In [ ]:
#pip install gensim==3.8.3


In [ ]:
#!pip install pyLDAvis


In [ ]:
#pip install spacy


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import nltk
import scipy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from autocorrect import Speller
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from gensim.models import CoherenceModel
import gensim
import gensim.corpora as corpora
import spacy

In [ ]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
!type -a pip

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:
merged = pd.read_csv("merged_new.csv")
merged.Content

In [ ]:
#display word count
merged['word_count'] = merged['Content'].apply(lambda x: len(str(x).split(" ")))
merged[['Content','word_count']].head()

# Data Prep

In [ ]:
#lowercase content
merged['Content'] = merged['Content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
merged['Content'].head()

In [ ]:
#remove punctuations
merged['Content'] = merged['Content'].str.replace('[^\w\s]','')
merged['Content'].head()

In [ ]:
#display number of stopwords

from nltk.corpus import stopwords
stop = stopwords.words('english')

merged['stopwords'] = merged['Content'].apply(lambda x: len([x for x in x.split() if x in stop]))
merged[['Content','stopwords']].head()

In [ ]:
print(stop)

In [ ]:
#remove stopwords
stop = stopwords.words('english')
merged['Content'] = merged['Content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
merged['Content'].head()

# Exploratory analysis 

In [ ]:
TextBlob(merged['Content'][0]).ngrams(2)

In [ ]:
TextBlob(merged['Content'][0]).ngrams(3)

In [ ]:
#words with highest frequency
freq = pd.Series(' '.join(merged['Content']).split()).value_counts()[:20]
freq

In [ ]:
#add new stop words

stop = stopwords.words('english')
newStopWords = ['also','1','2','3','4','eu','renewables','china','csfs','turkey','et','al','different','jordan','maan','anman','important','results','however','paper','new','implications','research','may','factors','study','success','this''csf','project','finding','result','decision','impact''important','solar','wind','renewable','energy','factors','projects','success','study']
stop.extend(newStopWords)

merged['Content'] = merged['Content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
merged['Content'].head()


In [ ]:
#words with highest frequency
freq = pd.Series(' '.join(merged['Content']).split()).value_counts()[:20]
freq

In [ ]:
TextBlob(merged['Content'][0]).ngrams(2)

In [ ]:
TextBlob(merged['Content'][0]).ngrams(3)

In [ ]:
print(stop)

# Data prep

In [ ]:
#tokenization
TextBlob(merged['Content'][1]).words

#lemmatization
from textblob import Word
merged['Content'] = merged['Content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
merged['Content'].head()

# Non-Negative Matrix Factorisation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=stop)
doc_term_matrix = tfidf_vect.fit_transform(merged['Content'].values.astype ('U'))

In [ ]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=5, random_state=42)
nmf.fit(doc_term_matrix)

In [ ]:
import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

In [ ]:
first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]

In [ ]:
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

In [ ]:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

In [ ]:
topic_values = nmf.transform(doc_term_matrix)
merged['Topic'] = topic_values.argmax(axis=1)
merged.head()